<a href="https://colab.research.google.com/github/claudiuan/Song-Recomandation/blob/main/Song_recomandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import modules**

In [ ]:
from google.colab import drive

import numpy as np
import pandas as pd

# **Loading Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
songDf1=pd.read_csv('/content/drive/MyDrive/Datasets/triplets_file2.csv')
songDf1.head()

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [ ]:
songDf2=pd.read_csv('/content/drive/MyDrive/Datasets/song_data2.csv')
songDf2.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [ ]:
#Merge them and delete duplicates by song_id
songDf=pd.merge(songDf1, songDf2.drop_duplicates(['song_id']), on='song_id', how='left')

songDf.head()
# 1m songs
# 2m info from users
len(songDf)

918949

# **Data preparation**

In [ ]:
# Merge title and artist_name
songDf['song'] = songDf['title'] + " - " + songDf['artist_name']

songDf.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,NaN,NaN,NaN,NaN,NaN
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007.0,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,NaN,NaN,NaN,NaN,NaN
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,NaN,NaN,NaN,NaN,NaN


In [ ]:

#song_df=song_df.head(10000)

In [ ]:
#cumulative sum of listen count of the songs
songGrouped=songDf.groupby(['song']).agg({'listen_count':'count'}).reset_index()

songGrouped.head()

,song,listen_count
0,#40 - DAVE MATTHEWS BAND,144
1,'97 Bonnie & Clyde - Eminem,39
2,(Anaesthesia) Pulling Teath - Metallica,38
3,(I Got That) Boom Boom - Britney Spears featur...,59
4,..Come Around - Collie Buddz,74


In [ ]:
groupedSum=songGrouped['listen_count'].sum()

songGrouped['percentage'] = (songGrouped['listen_count'] / groupedSum)*100
songGrouped.sort_values(['listen_count', 'song'], ascending=[0,1])

,song,listen_count,percentage
2925,Undo - Björk,3280,1.074314
681,Dog Days Are Over (Radio Edit) - Florence + Th...,3258,1.067109
450,Canada - Five Iron Frenzy,1587,0.519798
2391,Sincerité Et Jalousie - Alliance Ethnik,1435,0.470013
410,Bulletproof - La Roux,1392,0.455929
...,...,...,...
1530,Leave The Pieces (Album Version) - The Wreckers,22,0.007206
1867,No Creo En El Jamas - Juanes,22,0.007206
3051,What Your Soul Sings - Massive Attack,22,0.007206
743,Dónde jugarán los niños - Maná,19,0.006223


# **Popularity Recommandation Engine**

In [ ]:
class popularityRecommender():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None

    #Create the popularity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

        #Get a count of user_ids for each unique song as recommendation score
        train_data_grouped = train_data.groupby([self.item_id]).agg({self.user_id: 'count'}).reset_index()
        train_data_grouped.rename(columns = {'user_id': 'score'},inplace=True)

        #Sort the songs based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.item_id], ascending = [0,1])

        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')

        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(10)

    #Use the popularity based recommender system model to
    #make recommendations
    def recommend(self, user_id):
        user_recommendations = self.popularity_recommendations

        #Add user_id column for which the recommendations are being generated
        user_recommendations['user_id'] = user_id

        #Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]

        return user_recommendations

In [ ]:
pr =popularityRecommender()

pr.create(songDf, 'user_id', 'song')

In [ ]:
pr.recommend(songDf['user_id'][5])

,user_id,song,score,Rank
2925,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Undo - Björk,3280,1.0
681,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dog Days Are Over (Radio Edit) - Florence + Th...,3258,2.0
450,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Canada - Five Iron Frenzy,1587,3.0
2391,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Sincerité Et Jalousie - Alliance Ethnik,1435,4.0
410,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Bulletproof - La Roux,1392,5.0
2195,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Représente - Alliance Ethnik,1391,6.0
1627,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Lucky (Album Version) - Jason Mraz & Colbie Ca...,1343,7.0
1437,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Just Dance - Lady GaGa / Colby O'Donis,1335,8.0
84,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Ain't Misbehavin - Sam Cooke,1317,9.0
2440,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Somebody To Love - Justin Bieber,1274,10.0


# **Item Similarity Recommandation**

In [ ]:
#Class for Item similarity based Recommender System model
class itemSimilarityRecommender():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.cooccurence_matrix = None
        self.songs_dict = None
        self.rev_songs_dict = None
        self.item_similarity_recommendations = None

    #Get unique items (songs) corresponding to a given user
    def get_user_items(self, user):
        user_data = self.train_data[self.train_data[self.user_id] == user]
        user_items = list(user_data[self.item_id].unique())

        return user_items

    #Get unique users for a given item (song)
    def get_item_users(self, item):
        item_data = self.train_data[self.train_data[self.item_id] == item]
        item_users = set(item_data[self.user_id].unique())

        return item_users

    #Get unique items (songs) in the training data
    def get_all_items_train_data(self):
        all_items = list(self.train_data[self.item_id].unique())

        return all_items

    #Construct cooccurence matrix
    def construct_cooccurence_matrix(self, user_songs, all_songs):

        #Get users for all songs in user_songs.
        user_songs_users = []
        for i in range(0, len(user_songs)):
            user_songs_users.append(self.get_item_users(user_songs[i]))

        #Initialize the item cooccurence matrix of size
        #len(user_songs) X len(songs)
        cooccurence_matrix = np.matrix(np.zeros(shape=(len(user_songs), len(all_songs))), float)

        #Calculate similarity between user songs and all unique songs
        #in the training data
        for i in range(0,len(all_songs)):
            #Calculate unique listeners (users) of song (item) i
            songs_i_data = self.train_data[self.train_data[self.item_id] == all_songs[i]]
            users_i = set(songs_i_data[self.user_id].unique())

            for j in range(0,len(user_songs)):

                #Get unique listeners (users) of song (item) j
                users_j = user_songs_users[j]

                #Calculate intersection of listeners of songs i and j
                users_intersection = users_i.intersection(users_j)

                #Calculate cooccurence_matrix[i,j] as Jaccard Index
                if len(users_intersection) != 0:
                    #Calculate union of listeners of songs i and j
                    users_union = users_i.union(users_j)

                    cooccurence_matrix[j,i] = float(len(users_intersection))/float(len(users_union))
                else:
                    cooccurence_matrix[j,i] = 0


        return cooccurence_matrix


    #Use the cooccurence matrix to make top recommendations
    def generate_top_recommendations(self, user, cooccurence_matrix, all_songs, user_songs):
        print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(cooccurence_matrix))

        #Calculate a weighted average of the scores in cooccurence matrix for all user songs.
        user_sim_scores = cooccurence_matrix.sum(axis=0)/float(cooccurence_matrix.shape[0])
        user_sim_scores = np.array(user_sim_scores)[0].tolist()

        #Sort the indices of user_sim_scores based upon their value
        #Also maintain the corresponding score
        sort_index = sorted(((e,i) for i,e in enumerate(list(user_sim_scores))), reverse=True)

        #Create a dataframe from the following
        columns = ['user_id', 'song', 'score', 'rank']
        #index = np.arange(1) # array of numbers for the number of samples
        df = pd.DataFrame(columns=columns)

        #Fill the dataframe with top 10 item based recommendations
        rank = 1
        for i in range(0,len(sort_index)):
            if ~np.isnan(sort_index[i][0]) and all_songs[sort_index[i][1]] not in user_songs and rank <= 10:
                df.loc[len(df)]=[user,all_songs[sort_index[i][1]],sort_index[i][0],rank]
                rank = rank+1

        #Handle the case where there are no recommendations
        if df.shape[0] == 0:
            print("The current user has no songs for training the item similarity based recommendation model.")
            return -1
        else:
            return df

    #Create the item similarity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

    #Use the item similarity based recommender system model to
    #make recommendations
    def recommend(self, user):

        #A. Get all unique songs for this user
        user_songs = self.get_user_items(user)

        print("No. of unique songs for the user: %d" % len(user_songs))

        #B. Get all unique items (songs) in the training data
        all_songs = self.get_all_items_train_data()

        print("no. of unique songs in the training set: %d" % len(all_songs))

        #C. Construct item cooccurence matrix of size
        #len(user_songs) X len(songs)
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)

        #D. Use the cooccurence matrix to make recommendations
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)

        return df_recommendations

    #Get similar items to given items
    def get_similar_items(self, item_list):

        user_songs = item_list

        #B. Get all unique items (songs) in the training data
        all_songs = self.get_all_items_train_data()

        print("no. of unique songs in the training set: %d" % len(all_songs))

        #C. Construct item cooccurence matrix of size
        #len(user_songs) X len(songs)
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)

        #D. Use the cooccurence matrix to make recommendations
        user = ""
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)

        return df_recommendations

In [ ]:
ir = itemSimilarityRecommender()

ir.create(songDf, 'user_id', 'song')

In [ ]:
userItems = ir.get_user_items(songDf['user_id'][10])

In [ ]:
#for userItems in userItems:
#  print(userItems)

In [ ]:
listForRecomandations= ir.recommend(songDf['user_id'][5])

No. of unique songs for the user: 16
no. of unique songs in the training set: 3218
Non zero values in cooccurence_matrix :22486


In [ ]:

#import multiprocessing
#from functools import partial

#BUCKET_SIZE = 50000

#def run_process(start):
#  ab=ir.get_similar_items(['U Smile - Justin Bieber'])

#pool = multiprocessing.Pool()
#func = partial(run_process)
#pool.close()
#pool.join()


In [ ]:
ir.get_similar_items(['U Smile - Justin Bieber'])

no. of unique songs in the training set: 3218
Non zero values in cooccurence_matrix :2308


,user_id,song,score,rank
0,,Eenie Meenie - Sean Kingston and Justin Bieber,0.280702,1
1,,Love Me - Justin Bieber,0.271399,2
2,,Somebody To Love - Justin Bieber,0.261290,3
3,,Stuck In The Moment - Justin Bieber,0.213785,4
4,,One Time - Justin Bieber,0.213462,5
5,,Never Let You Go - Justin Bieber,0.177799,6
6,,Lucky (Album Version) - Jason Mraz & Colbie Ca...,0.118232,7
7,,The Only Exception (Album Version) - Paramore,0.117647,8
8,,Cosmic Love - Florence + The Machine,0.110969,9
9,,Bulletproof - La Roux,0.103834,10
